# 🗺️**Understanding Sustainable Development Goals through maps**  
## Exploring the role of map generalisation

The aim of this Notebook is to present some **map generalization techniques** (i.e. transforming the cartographic representation of entities to reduce map complexity), which are essential when mapping a phenomenon at different scales. 
- Some of the techniques presented are based on **CartAGen**, which is :

    - an **open source Python library** made by **IGN** (i.e the french national mapping agency) researchers
    - a tool to carry out cartographic generalization processes and **automate** them
    - a **QGIS plugin** to use the Python library in a GIS environment 

- 🔗 **Useful links** : 
    - [Library repository](https://github.com/LostInZoom/cartagen4py?tab=readme-ov-file) 
    - [QGIS Plugin](https://plugins.qgis.org/plugins/cartagen4qgis/#plugin-details)
    - [CartAGen documentation](https://cartagen.readthedocs.io/en/latest/)
    - [UN Sustainable Development Goals](https://sdgs.un.org/goals)

_by Paul Bourcier, 2025 - EUPL 1.2_ 

## ⚙️ **Part 0 :** Preparing the working environment

In [ ]:
# Importing libraries

import cartagen as cg # the main library, containing cartographic generalization algorithms

#geographic data manipulation 
import geopandas as gp # used to import and manipulate geographic data
from shapely.geometry import Polygon, MultiPolygon # manipulation of geometries
from shapely.ops import transform #

#displaying the data
from matplotlib import pyplot as plt # for ploting results, especially maps
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar # adding scalebar to the maps
#import matplotlib.colors as matcol # generating color palettes
#from matplotlib.path import Path  # Plot generation
#from matplotlib.patches import PathPatch # Plot generation
#from mpl_toolkits.axisartist.axislines import Subplot # Plot generation
#import matplotlib.colors as mcolors #generating colors 
#import matplotlib.cm as cm #used to display colorbar
import contextily as ctx # Adding basemaps
import mapclassify # Choropletes maps
import branca.colormap # linear color palettes for maps

#creating interactive outputs
import ipywidgets # creation of interactive plots
import pydeck as pdk #deck.gl in Python for interactive maps
from IPython.display import display, HTML #display interactive content
#import json #manipulating dictionnaries
import ipyleaflet #creates interactive maps

# other data manipulation
import pandas as pd # used to manipulate flat datas
import numpy as np # Using mathematical operations in Python
import json # manipulation of json files
#from sklearn import cluster #clustering methods

import warnings # Remove warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Numba not installed. Using slow pure python version.") # Remove user warning 


## 📈 **Part 1 :** Maps and SDGs : the example of the Gini index

🧠 To understand the SDGs and how to achieve them, we need to understand certain **indicators**. Let's take the example of SDG #10: _“Reduce inequality within and among countries”_.

The **Gini** index is a key indicator for understanding this goal, as it measures how a **resource** (notably disposable income) is **distributed** among the inhabitants of a country or region. It ranges from **0** (perfect equality) to **1** (no equality).

The Gini index is derived from the **Lorenz curve**. Before we dive into the maps, let's take a closer look.

### **1.1** Gini index and Lorenz curve principle

The code used in this section does mainly comes from this [webpage](https://zhiyzuo.github.io/Plot-Lorenz/)

In [ ]:
# First, we need to recover all the income of a population. Here, we construct a sample of artificial values, representing our income.
income = np.array([2, 10, 24, 4, 7, 800, 56, 32, 1, 5, 46, 78, 336])  
# Let's fill in our table of values using numpy functions, which allow us to obtain a larger sample more quickly.
income = np.append(np.random.poisson(lam=10, size=40), np.random.poisson(lam=100, size=10)) #Here, we use the Poisson random function for the draw. We could use another function

# Secondly, we must sort our income, from lowest to highest
income = np.sort(income)

# Third, we convert our values into cumulative proportions
income = income.cumsum() / income.sum()
# We then add 0 as the first value
income = np.insert(income,0,0)

# We now calculate the cumulative proportions of households in our population
pop = np.arange(income.size)/(income.size-1)

# Finally, we can display the Lorenz curve, which shows the distribution of cumulative incomes and households with these incomes.
fig, ax = plt.subplots(figsize=[6,6])
ax.scatter(pop , income, marker='x', color='darkgreen', s=100)
ax.plot([0,1], [0,1]) # We add a line representing perfect equality
ax.set_xlabel('Cumulative proportion of households')
ax.set_ylabel('Cumulative proportion of income')


📈 The x-axis represents the **cumulative proportion of households** (sorted by income), while the y-axis shows the **cumulative proportion of income**. In this situation, around **80%** of the poorest people hold less than **30%** of the income, which constitutes a clear inequality. In contrast, the straight line shows the **perfect equality** situation, where **80%** of the poorest hold **80%** of the income.

The Gini index corresponds to the area between the Lorenz curve and the straight line representing perfect equality. It can also be calculated using the following formula:
$$
Gini = \frac{2 \sum_{i=1}^{n} i \times y_i}{n \sum_{i=1}^{n} y_i} - \frac{n + 1}{n}
$$

This is a weighted sum of incomes, divided by the sum of incomes, and normalized by subtracting a constant.

In [ ]:
# Let's calculate the Gini index for our first example
coef_ = 2. / income.size # Numerator part 1
weighted_sum = sum([(i+1)*yi for i, yi in enumerate(income)]) # Numerator part 2
const_ = (income.size + 1.) / income.size # Constant

gini = coef_*weighted_sum/(income.sum()) - const_
print("Gini index : " + str(gini))

Here is an animation to illustrate the effect of the Lorenz curve on the Gini index

In [ ]:
# Create a function that generates and displays Lorenz curve and Gini index
def plt_gini_lorenz(income_diff):
    # Generate a sample based on the income_diff argument
    incm = np.append(np.random.poisson(lam=10, size=50), np.random.poisson(lam=income_diff, size=50)) 
    incm = np.sort(incm) # Sort the sample of income
    incm = incm.cumsum() / incm.sum() # Convert it to proportions
    incm = np.insert(incm,0,0) # Add a zero to our list
    popu = np.arange(incm.size)/(incm.size-1) # Calculate a list of proportions of household

    # Calculate Gini index based on the formula
    gini = (2. / incm.size)*sum([(i+1)*yi for i, yi in enumerate(incm)])/(incm.sum()) - ((incm.size + 1.) / incm.size) 

    # Display the curve and the index
    fig, ax = plt.subplots(figsize=[6,6])
    ax.scatter(popu , incm, marker='x', color='darkgreen', s=100) 
    ax.plot([0,1], [0,1], color='k')
    ax.text(0.35, 0.8, f'Gini index : {round(gini,2)}', fontsize=12, color='red', ha='right')

# Execute function with ipywidgets to enable interactivity
ipywidgets.interact(plt_gini_lorenz, income_diff = (0, 100, 1))

### **1.2** Spatializing the Gini index

As a powerful indicator for measuring inequality, the Gini index is relevant for comparing situations between countries.

Maps are well suited to quickly perform that comparison, by spatializing the index.

In [ ]:
# Create and display a simple choropleth map
gini_world = gp.read_file("data/gini_world.geojson") # Load the data (i.e world country with a 'gini_index' column)

ax = gini_world.plot(linewidth=0.5, zorder = 1, cmap = 'YlOrRd',
                     edgecolor = 'black', column = 'gini_index', legend = True,
                     alpha = 0.8, scheme='FisherJenks',
                     legend_kwds ={'loc':'upper left','title': 'Gini index (most recent)'}, figsize= (12,12)) # Main layer

# Optional elements
ax.axes.get_xaxis().set_visible(False) # Remove x axis
ax.axes.get_yaxis().set_visible(False) # Remove y axis

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron) # Basemap
plt.title('Gini index in the world countries\n') # Title

This map allows you to quickly compare the Gini index **across countries**. But the question of inequality does not only arise at **international level.** 

To show the spatiality of a phenomenon at different **scales**, cartographers have two options:
- produce several maps at each chosen scale (**atlas**)
- create a map that encompasses all the desired scales (**multi-scale map**).

In this Notebook, we have chosen the second option

### **1.3** Gini index at local scale

In this section, we will be using data from INSEE (a French public statistics institute), which includes Gini indices at the IRIS level (i.e. the smallest level available in France).

*Unfortunately, Gini index values are only available for IRIS with more than 5,000 inhabitants.*

In [ ]:
#Load the data as a json object
gini_iris = open('data/gini_local.geojson') # Geographic data must be in .geojson format
gini_iris = json.load(gini_iris)

We are now going to build our first **multi-scale interactive map**. In this Notebook, we have chosen the **pydeck** library to do this.

✔️It has the advantage of producing very **fluid** renderings (using vector tile technology).

❌The drawback is that there are not many functions for **personalizing** the map.

In [ ]:
# Let's use the pydeck library to create an interactive map
view_state = pdk.ViewState(longitude=2.354, latitude=48.865, zoom=13, bearing=0, pitch=0) # Position the initial view in the middle of Paris

iris = pdk.Layer( # Creating the pydeck layer containing the IRIS  
    'GeoJsonLayer', 
    gini_iris, # Json/dict containing the data
    opacity=0.8,
    stroked=True,
    filled=True,
    get_fill_color=['properties.gini_index < 0.25 ?  255 :'+
    'properties.gini_index >= 0.25 && properties.gini_index < 0.3 ? 186 :'+
    'properties.gini_index >= 0.3 && properties.gini_index < 0.4 ? 128 :'+ 
    'properties.gini_index >= 0.4 && properties.gini_index < 0.55 ? 64 :0,'+
    'properties.gini_index < 0.25 ?  255 :'+
    'properties.gini_index >= 0.25 && properties.gini_index < 0.3 ? 186 :'+
    'properties.gini_index >= 0.3 && properties.gini_index < 0.4 ? 128 :'+ 
    'properties.gini_index >= 0.4 && properties.gini_index < 0.55 ? 64 :0, 255'], 
    # Here we use a javascript expression to manually create classes for fill colors
    get_line_color=[0, 0, 0],
    get_line_width = 10,
    pickable=True
)

# Creating the map object
r = pdk.Deck(layers=[iris], initial_view_state=view_state, tooltip={"text": "{properties.gini_index}\n{properties.iris_name}"}) 

map1 = r.to_html()  # Store the map as a HTML object

⚙️ Pydeck does not natively offer functions for creating a **legend**. We therefore create a legend by directly modifying the HTML code of our map

In [ ]:
# # Retrieve the content of out the HTML map as a string
# map1_str = map1.data 

# # Prepare the HTML code of our legend
# legend_html = """
# <div style="position: absolute; top: 20px; left: 20px; background-color: rgba(255, 255, 255, 0.8); padding: 10px; border-radius: 5px; font-family: Arial, sans-serif; font-size: 14px; color:rgb(10, 10, 10); z-index: 10;">
#   <div style="display: flex; align-items: center; margin-bottom: 5px;">
#     <div style="width: 20px; height: 20px; background-color: rgb(255, 255, 255); margin-right: 5px;"></div>
#     <span>Gini Index < 0.25</span>
#   </div>
#   <div style="display: flex; align-items: center; margin-bottom: 5px;">
#     <div style="width: 20px; height: 20px; background-color: rgb(186, 186, 255); margin-right: 5px;"></div>
#     <span>0.25 <= Gini Index < 0.3</span>
#   </div>
#   <div style="display: flex; align-items: center; margin-bottom: 5px;">
#     <div style="width: 20px; height: 20px; background-color: rgb(128, 128, 255); margin-right: 5px;"></div>
#     <span>0.3 <= Gini Index < 0.4</span>
#   </div>
#   <div style="display: flex; align-items: center; margin-bottom: 5px;">
#     <div style="width: 20px; height: 20px; background-color: rgb(64, 64, 255); margin-right: 5px;"></div>
#     <span>0.4 <= Gini Index < 0.55</span>
#   </div>
#   <div style="display: flex; align-items: center; margin-bottom: 5px;">
#     <div style="width: 20px; height: 20px; background-color: rgb(0, 0, 255); margin-right: 5px;"></div>
#     <span>Gini Index >= 0.55</span>
#   </div>
# </div>
# """

# # Bind and display the legend and the map
# display(HTML(map1_str + legend_html))

👓 This map is satisfactory for exploring the spatial distribution of the Gini index **across a city**. Taking Paris as an example, we can clearly see that the **center and west** of the city are less egalitarian in terms of living standards.

However, zooming out to a broader level leads to problems of **legibility**. IRIS boundaries become too small, and the number of IRIS displayed tends to **overload** the map. In this case, the map needs to be **generalized**.

## 🛠 **Part 2 :** Map generalisation and multi-scale maps

Reducing the **graphical complexity** of a map is essential to enable an efficient understanding of its **message**. It this Notebook, we propose **two** approaches to generalize our local gini index map. 

### **2.1** Using one generalisation technique

In this section, we will perform a generalisation method that consists in aggregating spatial entities in a regular grid.

#### **2.1.1** Principle 

In [ ]:
# Load the IRIS file as a GeoDataFrame and change to metric projection
geo_gini_iris = gp.read_file('data/gini_local.geojson')
geo_gini_iris.to_crs('EPSG:3857',inplace = True) 

# Calculte the centroid of each IRIS polygon
for i in range(len(geo_gini_iris)):
    geo_gini_iris.loc[i,"centroid_coords"] = geo_gini_iris.loc[i,"geometry"].centroid

# Copy this layer and set the centroids as the geometry
point_gini_iris = geo_gini_iris.copy()
point_gini_iris = point_gini_iris.set_geometry("centroid_coords")
point_gini_iris = point_gini_iris.drop(columns=['geometry'])
point_gini_iris.rename_geometry("geometry", inplace = True)

We are now ready to perform the aggregation !

In [ ]:
# This is the code line to use Caratagen labelgrid_aggregation algorithm, which return a grid (as a GeoDataFrame) containing mean gini index values within each cell
# (it also return a GeoDataFrame of points representing the center of each cell. We do not use it here).

#point_agr_1 = cg.labelgrid_aggregation(point_gini_iris, 1500, 1500, column='gini_index', shape='square', grid=True)

# As the computing time is very long because of the size of our dataset, we will load the result directly
point_agr_1 = gp.read_file('data/point_agr_1.geojson')

# Once loaded, we remove cell without values (i.e where no point was found)
point_agr_1 = point_agr_1.dropna()
# We then set the crs of our layer to 4326, as it is later needed to display it in pydeck
point_agr_1.to_crs('EPSG:4326',inplace = True)

# Export the layer as a .geojson file to open it in pydeck...
point_agr_1.to_file("point_agr_1_ok.geojson") 
# ...And load it as a json object
gini_agr_1 = open('point_agr_1_ok.geojson')
gini_agr_1 = json.load(gini_agr_1)


In [ ]:
# Let's use the pydeck library to create our 2nd interactive map
view_state = pdk.ViewState(longitude=2.354, latitude=48.865, zoom=11, bearing=0, pitch=0) 

iris = pdk.Layer( # Creating the pydeck layer containing the IRIS  
    'GeoJsonLayer', 
    gini_agr_1, # This time we load our aggregated data
    opacity=0.8, stroked=True, filled=True, get_fill_color=['properties.gini_index_mean < 0.25 ?  255 :'+ 'properties.gini_index_mean >= 0.25 && properties.gini_index_mean < 0.3 ? 186 :'+ 'properties.gini_index_mean >= 0.3 && properties.gini_index_mean < 0.4 ? 128 :'+ 'properties.gini_index_mean >= 0.4 && properties.gini_index_mean < 0.55 ? 64 :0,'+'properties.gini_index_mean < 0.25 ?  255 :'+'properties.gini_index_mean >= 0.25 && properties.gini_index_mean < 0.3 ? 186 :'+'properties.gini_index_mean >= 0.3 && properties.gini_index_mean < 0.4 ? 128 :'+ 'properties.gini_index_mean >= 0.4 && properties.gini_index_mean < 0.55 ? 64 :0, 255'], get_line_color=[0, 0, 0],get_line_width = 25,pickable=True
)

# Creating the map object
r = pdk.Deck(layers=[iris], initial_view_state=view_state, tooltip={"text": "mean Gini index : {properties.gini_index_mean}"}) 

r.to_html()  # Directly display the result

With this more generalized representation of our data, it is easier to understand the distribution of the Gini index at a smaller scale.
However, as the cells of the grid do not follow the administrative boundaries and hide the background map, it can be difficult to see where we are.

Let's add a layer representing commune boundaries to our map

In [ ]:
#Load the data as a json object
communes_boundaries = open('data/communes_5.geojson') # Geographic data must be in .geojson format
communes_boundaries = json.load(communes_boundaries)

# We load another administrative layer, used later 
departments_boundaries = open('data/departements.geojson')
departments_boundaries = json.load(departments_boundaries)

In [ ]:
# 2nd map with administrative boundaries
view_state = pdk.ViewState(longitude=2.354, latitude=48.865, zoom=11, bearing=0, pitch=0) # Position the initial view in the middle of Paris

iris = pdk.Layer( 'GeoJsonLayer', gini_agr_1, opacity=0.8,stroked=True,filled=True,get_fill_color=['properties.gini_index_mean < 0.25 ?  255 :'+'properties.gini_index_mean >= 0.25 && properties.gini_index_mean < 0.3 ? 186 :'+'properties.gini_index_mean >= 0.3 && properties.gini_index_mean < 0.4 ? 128 :'+ 'properties.gini_index_mean >= 0.4 && properties.gini_index_mean < 0.55 ? 64 :0,'+'properties.gini_index_mean < 0.25 ?  255 :'+'properties.gini_index_mean >= 0.25 && properties.gini_index_mean < 0.3 ? 186 :'+'properties.gini_index_mean >= 0.3 && properties.gini_index_mean < 0.4 ? 128 :'+ 'properties.gini_index_mean >= 0.4 && properties.gini_index_mean < 0.55 ? 64 :0, 255'],  get_line_color=[0, 0, 0],get_line_width = 25,pickable=True)

communes = pdk.Layer( # Creating the pydeck layer containing the communes boundaries 
    'GeoJsonLayer', 
    communes_boundaries, # Json/dict containing the data
    stroked=True,
    get_line_color=[255, 0, 0], # Red line
    get_fill_color = [255,255,255,0], # Fill color with 0% of opacity, so we only see lines
    get_line_width = 50
)

# Creating the map object
r = pdk.Deck(layers=[iris, communes], initial_view_state=view_state, tooltip={"text": "mean Gini index : {properties.gini_index_mean}"}) 

r.to_html()  # Store the map as a HTML object

It is now way easier to undestand where the location of the map view is. This level of generalisation is good for comparing situation between several cities. 


Looking at Paris and the surrounding cities, we can see that inequalities are less important out of the capital, except some for communes in the west.

#### **2.1.2** Multi-scale map of the Gini index using grid aggregation method

We now want to create another even more generalised layer for our maps, to enable the reading of Gini index at smaller scale.

We then combine those different graphical representation of our data in our multi-scale map.

In [ ]:
#point_agr_2 = cg.labelgrid_aggregation(point_gini_iris, 6500, 6500, column='gini_index', shape='square', grid=True)

# Again, let's directly import aggregated layer 
point_agr_2 = gp.read_file('data/point_agr_2.geojson')
# Once loaded, we remove cell without values (i.e where no point was found)
point_agr_2 = point_agr_2.dropna()
# We then set the crs of our layer to 4326, as it is later needed to display it in pydeck
point_agr_2.to_crs('EPSG:4326',inplace = True)

# Export the layer as a .geojson file to open it in pydeck...
point_agr_2.to_file("point_agr_2_ok.geojson") 
# ...And load it as a json object
gini_agr_2 = open('point_agr_2_ok.geojson')
gini_agr_2 = json.load(gini_agr_2)

Unfortunatly, pydeck doesn't provide visibility control of layers depending on zoom level. Thus, we choose to create our map with the ipyleaflet library.

In [ ]:
# layer preparation
# 1: open the layer of the IRIS as a json file
gini_local = open('data/gini_local.geojson')
gini_local = json.load(gini_local)

# 2: extract the iris code and the gini index to form a dictionnary c
lst_index = []
for i in gini_local['features']:
    lst_index.append(i['properties']['gini_index'])

lst_id = []
for i in gini_local['features']:
    lst_id.append(i['properties']['iris_code'])

gini_data = dict(zip(lst_id, lst_index))

# 3: modify the IRIS json to add the iris code within the features
for id, dico  in enumerate(gini_local['features']):
    gini_local['features'][id].update({'id':dico['properties']['iris_code']})

In [ ]:
# We perform the same processings for the first aggregated grid
lst_index = []
for i in gini_agr_1['features']:
    lst_index.append(i['properties']['gini_index_mean'])
                                               
lst_id = []
for id, dico in enumerate(gini_agr_1['features']):
    lst_id.append(id)

gini_data_2 = dict(zip(lst_id, lst_index))

for id, dico  in enumerate(gini_agr_1['features']):
    gini_agr_1['features'][id].update({'id':id})

In [ ]:
# We perform the same processings for the second aggregated grid
lst_index = []
for i in gini_agr_2['features']:
    lst_index.append(i['properties']['gini_index_mean'])
                                               
lst_id = []
for id, dico in enumerate(gini_agr_2['features']):
    lst_id.append(id)

gini_data_3 = dict(zip(lst_id, lst_index))

for id, dico  in enumerate(gini_agr_2['features']):
    gini_agr_2['features'][id].update({'id':id})

Our data are ready, let's build and display the multi-scale map ! (loading time can be up to 1 minute)

In [ ]:
# 1: Create the map object
m = ipyleaflet.Map(center=(48.8566, 2.3522), zoom=16,min_zoom =9, max_zoom = 18, basemap=ipyleaflet.basemaps.CartoDB.DarkMatter)


# 2: Configure the color palette 
breaks = [0, 0.25, 0.3, 0.4, 0.55, 1]  
colormap = branca.colormap.linear.Blues_09.scale(0.163, 0.784).to_step(index=breaks)


# 3: Load datas as ipyleaflet layers
iris_layer = ipyleaflet.Choropleth(
    geo_data=gini_local, 
    choro_data=gini_data, # Geographic and thematic data are separated in the Choropleth class 
    key_on = 'id', # This id is used to link geo_data and choro_data
    colormap=colormap,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

grid_layer = ipyleaflet.Choropleth(geo_data=gini_agr_1, choro_data=gini_data_2, key_on = 'id', colormap=colormap, border_color='black', style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

grid2_layer = ipyleaflet.Choropleth(geo_data=gini_agr_2, choro_data=gini_data_3, key_on = 'id', colormap=colormap, border_color='black', style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

communes = ipyleaflet.GeoJSON(data=communes_boundaries, style={'opacity': 1, 'fillOpacity': 0, 'color':'red', 'weight':1}) # Load the administrative boundaries as well
dep = ipyleaflet.GeoJSON(data=departments_boundaries,style={'opacity': 1, 'fillOpacity': 0, 'color':'red', 'weight':1})


# 4: Create a function to load/unload layer on the map, depending on zoom level
def on_zoom_change(event):
    if m.zoom >= 13: 
        if len(m.layers[1].choro_data) < 10000: # Verify if the loaded layer is the NOT one with more than 10 000 entities (i.e the iris layer)
            if len(m.layers) > 2: 
                m.remove_layer(m.layers[1])
                m.remove_layer(m.layers[1])
            else:
                m.remove_layer(m.layers[1])
            m.add(iris_layer) 
  
    elif m.zoom < 13 and m.zoom > 10:
        if len(m.layers[1].choro_data) < 2000 or len(m.layers[1].choro_data) > 10000: # Verify if the loaded layer is the NOT the intermediary grid layer
            if len(m.layers) > 2:
                m.remove_layer(m.layers[1])
                m.remove_layer(m.layers[1])
            else:
                m.remove_layer(m.layers[1])
            m.add(grid_layer) 
            m.add(communes)
    else:
        if len(m.layers[1].choro_data) > 2000:  # Verify if the loaded layer is the NOT the second grid layer
            m.remove_layer(m.layers[1])
            m.remove_layer(m.layers[1])
            m.add(grid2_layer)
            m.add(dep)
         
# 5: Use the ipyleaflet method to detect when a zoom is performed
m.observe(on_zoom_change, names='zoom')

# 6: add the iris layer and display the map
m.add(iris_layer) 
display(m)